# Simulated Annealing for a 3D Skyrmion Lattice

## TODOS

* Recheck computations (signs in delta energy, acceptProb, ...)
* Benchmark: padded grid, always use mod for indices
* Read and think about parallelization
* Setup hdf5 output of complete configuration at snapshots
* Put analysis in place: autocorrelation, phase diagram, etc.
* Best way to setup parameterfile that i can control from the makefile?

In [ ]:
using PyCall
using PyPlot
using HDF5
using StatsBase
# using ParallelAccelerator
# using Distributions
using Base.Profile
using Base.Test
using ProgressMeter
# @pyimport matplotlib.animation as anim

## Parameters & Initialization

In [ ]:
srand(123);

### Type Definitions and Initialization

In [ ]:
type Parameters
    N::Int64 # number of lattice points in each direction
    Ntot::Int64 # total number of lattice points
    Nsweeps::Int64 # number of sweeps
    Nconfig::Int64 # number of configurations to save
    T::Float64 # the current temperature
    Tmu::Float64 # the reduction factor of the temperature
    Tmin::Float64 # the minimal temperature at which we stop
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # magnetic field in 3 direction
    function Parameters(; N::Int64=10, Nsweeps::Int64=30, Nconfig::Int64=100, T::Float64=1.0, Tmu::Float64=1.05, Tmin::Float64=1.0e-8, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0)
        Ntot = N^3
        Jp = J/16.0
        Kp = K/8.0
        new(N, Ntot, Nsweeps, Nconfig, T, Tmu, Tmin, J, Jp, K, Kp, B)
    end
end

In [ ]:
type Monitor
    reject::Int64 # number of rejected steps
    accept::Int64 # number of accepted steps
    Monitor(; reject::Int64=0, accept::Int64=0) = new(reject, accept)
end

In [ ]:
type Results
    E::Vector{Float64} # total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # temperature
    idx::Int64 # current index (monte carlo time)
    Results(pars::Parameters) = new(zeros(Float64, pars.Nconfig+1), 0.0, zeros(Float64, 3, pars.Nconfig+1), zeros(Float64, 3), zeros(Float64, pars.Nconfig+1), 0)
end

In [ ]:
function init(pars::Parameters)
    (Monitor(), Results(pars))
end

function init()
    pars = Parameters()
    (pars, init(pars)...)
end

### Grid Initialization

In [ ]:
function uniformS2(dims::Tuple{Int64,Int64,Int64})
    phi = 2pi .* rand(dims)
    u = 2rand(dims) - 1.0
    squ = sqrt(1.0 - u.*u)
    x = squ .* cos(phi)
    y = squ .* sin(phi)
    res = zeros(3,dims...)
    for i=1:*(dims...)
        @inbounds res[:,i] = [x[i] y[i] u[i]]
    end
    res
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi*rand()
    u = 2rand() - 1.0
    squ = sqrt(1.0 - u*u)
    s[1] = squ*cos(phi)
    s[2] = squ*sin(phi)
    s[3] = u
    nothing
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64,Int64,Int64})
    grid = uniformS2(dims)
    enforcePeriodicity!(grid)
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    info("Initialized random grid.")
    grid
end

function initRandomGrid()
    L = pars.N+2
    initRandomGrid((L,L,L))
end

## Grid Manipulations

In [ ]:
function enforcePeriodicity!(grid::Array{Float64,4})
    N = pars.N::Int64
    for i=1:2
        grid[:,N+i,:,:] = grid[:,i,:,:]
        grid[:,:,N+i,:] = grid[:,:,i,:]
        grid[:,:,:,N+i] = grid[:,:,:,i]
    end
    nothing
end

function enforcePeriodicity!(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64)
    N = pars.N::Int64
    ii = false; jj = false
    if i < 3
        grid[:,i+N,j,k] = grid[:,i,j,k]
        ii = true
    end
    if j < 3
        grid[:,i,j+N,k] = grid[:,i,j,k]
        ii && (grid[:,i+N,j+N,k] = grid[:,i,j,k])
        jj = true
    end
    if k < 3
        grid[:,i,j,k+N] = grid[:,i,j,k]
        ii && (grid[:,i+N,j,k+N] = grid[:,i,j,k])
        jj && (grid[:,i,j+N,k+N] = grid[:,i,j,k])
        ii && jj && (grid[:,i+N,j+N,k+N] = grid[:,i,j,k])
    end
end

In [ ]:
function isPeriodic(grid::Array{Float64,4})
    N = pars.N::Int64
    if grid[:,N+1,:,:] != grid[:,1,:,:] || grid[:,:,N+1,:] != grid[:,:,1,:] ||
       grid[:,:,:,N+1] != grid[:,:,:,1] || grid[:,N+2,:,:] != grid[:,2,:,:] ||
       grid[:,:,N+2,:] != grid[:,:,2,:] || grid[:,:,:,N+2] != grid[:,:,:,2]
        return false
    end
    true
end

In [ ]:
@inline function normalize!(grid::Array{Float64,4}, idx::Int64)
    grid[:,idx] /= norm(grid[:,idx])
end

function normalize!(grid::Array{Float64,4})
    for idx=1:pars.Ntot::Int64
        @inbounds normalize!(grid, idx)
    end
end

## Hamiltonian

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
@inline function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    crossX(grid[:,i,j,k], grid[:,i+os,j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,j+os,k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,k+os])
end

In [ ]:
function totalEnergy(grid::Array{Float64,4})
    N = pars.N::Int64
    E = 0.0
    for k=1:N, j=1:N, i=1:N
        E += - pars.B * grid[3,i,j,k] -
             pars.J  * dot(grid[:,i,j,k], grid[:,i+1,j,k] + grid[:,i,j+1,k] + grid[:,i,j,k+1]) +
             pars.Jp * dot(grid[:,i,j,k], grid[:,i+2,j,k] + grid[:,i,j+2,k] + grid[:,i,j,k+2]) -
             pars.K  * DMterm(grid, i, j, k, 1) +
             pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    N = pars.N::Int64
    M = zeros(Float64, 3)
    for k=1:N, j=1:N, i=1:N
        @inbounds M += grid[:,i,j,k]
    end
    M
end

In [ ]:
function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    N = pars.N::Int64
    sdiff = snew - grid[:,i,j,k]
    im1 = mod(i-2,N)+1; im2 = mod(i-3,N)+1
    jm1 = mod(j-2,N)+1; jm2 = mod(j-3,N)+1
    km1 = mod(k-2,N)+1; km2 = mod(k-3,N)+1
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, grid[:,im1,j,k] + grid[:,i+1,j,k] + grid[:,i,jm1,k] + grid[:,i,j+1,k] + grid[:,i,j,km1] + grid[:,i,j,k+1]) +
    pars.Jp::Float64 * dot(sdiff, grid[:,im2,j,k] + grid[:,i+2,j,k] + grid[:,i,jm2,k] + grid[:,i,j+2,k] + grid[:,i,j,km2] + grid[:,i,j,k+2]) -
    pars.K::Float64  * (crossX(sdiff, grid[:,i+1,j,k] - grid[:,im1,j,k]) +
                        crossY(sdiff, grid[:,i,j+1,k] - grid[:,i,jm1,k]) +
                        crossZ(sdiff, grid[:,i,j,k+1] - grid[:,i,j,km1])) +
    pars.Kp::Float64 * (crossX(sdiff, grid[:,i+2,j,k] - grid[:,im2,j,k]) +
                        crossY(sdiff, grid[:,i,j+2,k] - grid[:,i,jm2,k]) +
                        crossZ(sdiff, grid[:,i,j,k+2] - grid[:,i,j,km2]))
end

## Simulated Annealing

In [ ]:
@inline stepT!() = (pars.T::Float64 /= pars.Tmu::Float64)

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.accept::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += snew - grid[:,i,j,k]
    grid[:,i,j,k] = snew
    enforcePeriodicity!(grid, i, j, k)
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i,j,k = rand(1:pars.N::Int64, 3)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.reject::Int64 += 1
    end
end

In [ ]:
@inline function sweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot::Int64
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:pars.Nsweeps::Int64
        sweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    snapshot()
    @showprogress 1 "Thermalizing..." for i=1:pars.Nconfig::Int64
        sweep!(grid, snew)
        snapshot()
    end
    grid
end

function thermalize()
    grid = initRandomGrid()
    thermalize!(grid)
end

In [ ]:
function run!(grid::Array{Float64,4})
    snapshot()
    @showprogress 1 "Main run..." for i = 1:pars.Nconfig::Int64
        nextconfig!(grid)
        snapshot()
        stepT!()
        pars.T < pars.Tmin::Float64 && (info("Cooled below minimal temperature: $(pars.Tmin)"); break)
    end
    grid
end

function run()
    grid = initRandomGrid()
    run!(grid)
end

## Output

In [ ]:
function snapshot()
    res.idx::Int64 += 1
    res.E[res.idx] = res.Etmp::Float64
    res.M[:,res.idx] = res.Mtmp::Vector{Float64}
    res.T[res.idx] = pars.T::Float64
end

In [ ]:
function initIO(filename::ASCIIString)
    h5open(getPath(filename), "w") do file
        d_create(file, "configs", datatype(Float64), dataspace(3,pars.N,pars.N,pars.N,pars.Nconfig),"chunk",(3,pars.N,pars.N,pars.N,1)
    end
end

In [ ]:
function addConfig(grid::Array{Float64,4}, idx::Int64, filename::ASCIIString)
    h5open(getPath(filename), "w") do file
        dset = file["configs"]
        dset[:,:,:,:,idx] = grid
    end
end

In [ ]:
@inline function getPath(filename::ASCIIString)
    string("../../data/", filename, ".h5")
end

In [ ]:
@inline function writeArray(data::Array{Float64}, filename::ASCIIString, dataset::ASCIIString=filename)
    h5write(getPath(filename), dataset, data)
end

In [ ]:
@inline function writeGrid(grid::Array{Float64,4}, filename::ASCIIString)
    writeArray(grid[:, 1:pars.N, 1:pars.N, 1:pars.N], filename, "grid")
end

## Postprocessing

### Basic Statistics

In [ ]:
function variance_meansq(f::Array{Float64}; mean=nothing, meansq=nothing)
    if meansq == nothing
        if mean == nothing
            mean::Float64 = Base.mean(f)
        end
        meansq = mean*mean
    end
    (sumabs2(f)/length(f) - meansq, meansq)
end

In [ ]:
function autocorr(f::Vector{Float64})
    meansq, var = variance_meansq(f)
    N = length(f)
    auto = zeros(Float64, N)
    for t=0:N-1
        tmp = 0.0
        for i=1:N-t
            tmp += f[i]*f[i+t]
        end
        auto[t+1] = (tmp/(N-t) - meansq) / var
    end
    auto
end

In [ ]:
function intAutoTime(f::Vector{Float64}, N::Int64; isautocorr=false)
    if !isautocorr
        f = autocorr(f)
    end
    0.5 + sum(f[2:N])
end

function intAutoTime(f::Vector{Float64}; isautocorr=false)
    intAutoTime(f, length(f), isautocorr=isautocorr)
end

In [ ]:
# experimental, still flawed
function findIntAutoTime(f::Vector{Float64})
    auto = autocorr(f)
    I = 2:fld(length(f),5)
    ts = zeros(size(I))
    for (i, tau) in enumerate(I)
        ts[i] = intAutoTime(auto, 5tau, isautocorr=true)
    end
    val, idx = findmin(abs(ts-tau))
    (idx, ts)
end

### Skyrmion Specifics

In [ ]:
@inline getX(grid::Array{Float64}) = squeeze(grid[1,:],1)
@inline getY(grid::Array{Float64}) = squeeze(grid[2,:],1)
@inline getZ(grid::Array{Float64}) = squeeze(grid[3,:],1)

In [ ]:
@inline averageSpins(M::Array{Float64,2}) = M/pars.Ntot
@inline averageSpins!(M::Array{Float64,2}) = (M /= pars.Ntot; nothing)

In [ ]:
function avgConfigs()
end

In [ ]:
function braggIntensity(S::Array{Float64,2})
end

In [ ]:
function fft3(f::Array{Float64,3}, N::Int64)
    FFTW.set_num_threads(4)
    fft(f,(N, N, N))
end

## Run the Simulation

In [ ]:
pars = Parameters(N=10, Nsweeps=30, Nconfig=10000, T=0.0)
mon, res = init(pars);
pars.T, pars.B, pars.J, pars.K, pars.N, pars.Nconfig

In [ ]:
@time grid = thermalize();

In [ ]:
writeGrid(grid, "T0_N10")

In [ ]:
tMC = 1:pars.Nconfig+1
semilogx(tMC, res.E)
figure()
plot(tMC, getX(res.M), tMC, getY(res.M), tMC, getZ(res.M))

In [ ]:
plot(autocor(res.E, 0:length(res.E)-1))

In [ ]:
# Profile.clear()
# Profile.init()
# @time run();

In [ ]:
# data, lidict = Profile.retrieve();

## Tests

In [ ]:
function testuniformS2()
    N = 50
    dims = (N, N, N)
    grid = uniformS2(dims);
    for i=1:*(dims...)
        @test norm(grid[:,i]) ≈ 1.0
    end
    println("finished test: uniform points on S2")
end

In [ ]:
function test()
    testuniformS2()
    println("If you do not see any errors, all tests passed.")
end
@time test()